# Contents

### Version 1.0
0. Resources Used
    0.1 Demonstration
    0.2 Mathematics
1. What is reddit?
    1.1 Why reddit for a chatbot?
2. The Dataset

Notes to self:

Make a database to BUFFER the data. It's SO big that we can't just read it into our puny 32GB of RAM for our training set. Even just a month is big data (Reddit is massive). For SQLite3, we prepare a lot of pre-defined functions that insert themselves as SQL commands on a big database. (One provided by python-programing below)** Help! Dr. Richardson!

We're going to train our data using a Deep belief net using the theory on [neuro-machine translations](https://github.com/tensorflow/nmt) using something called "attention mechanisms" something related to [Long-Short Term Memory networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/).  ** Help! Dr. Richardson!

LSTMs are can remember decently sequences of tokens up to 10-20 in length fairly well. After, this point, their performance drops. For some reason a "Bidirectional" recurrent neural network does pretty well.

# 0 - Resources Used

  *[SQLite3](https://docs.python.org/2/library/sqlite3.html#)
  *[Pythonprogramming.net Reddit Chatbot](https://pythonprogramming.net/bidirectional-attention-mechanism-chatbot-deep-learning-python-tensorflow/?completed=/training-model-chatbot-deep-learning-python-tensorflow/) 
  *[SQL](https://pythonprogramming.net/mysql-intro/)
  *[Neuro-Machine Translator](https://github.com/tensorflow/nmt)
  *[open function](https://docs.python.org/2/library/functions.html#open)
  
### About SQLite3 
      SQLite is a C library that provides a lightweight disk-based database that doesn’t require a separate server process and allows accessing the database using a nonstandard variant of the SQL query language. Some applications can use SQLite for internal data storage. It’s also possible to prototype an application using SQLite and then port the code to a larger database such as PostgreSQL or Oracle.
  

## 0.1 The Deep-Learning Reddit Chatbot


In [ ]:
#insert finished code here. Make it collapsable.

# 1 What is reddit?

[Reddit](https://www.redditinc.com/) is a massive social-medial platforms on the internet, and it is famous for basically being the 'forum of forums'. Reddit is fractured into diverse user-made subreddits, which is a sub-forum for any niche topic). 

What makes reddit specifically good for a chatbot are a few reasons.

## Why Reddit for a chatbot?

    1) Reddit structures original comments and responses in a tree-like format, allowing it to store massive amounts of human-generated responses to human-generated statement and questions.

    2) Reddit uses 2 metrics to denote a somewhat "general" sense of value in a comment: voting and gold. Voting is when another user decides to evaluate your comment as positive {+1} or negative {-1}. Gold is when another person has purchased real money to show that your comment is important for some reason. Keep in mind that certain subreddits have different connotations as to what these mean.

    3) Because reddit is such a wealth of statement-> responses, we should be able to use a Recurrent Neural Networks model to look for the hidden context between sentences in a statement -> response format. (hidden markov fields to make an "intelligent" chatbot.)


### Interesting note
Certain subreddits may hold a unique micro-culture and conversational styles that may be useful for our chat-bot.


# The Dataset 

[A famous reddit post](https://www.reddit.com/r/datasets/comments/3bxlg7/i_have_every_publicly_available_reddit_comment/?st=j9udbxta&sh=69e4fee7) has publically made avaliable 1.7 billion reddit comments compressed as 250GBs of data, which is nice considering Reddit API under PRAW and scraping data individually is not worth the hassle.


Another user was kind enough to [sort](https://www.reddit.com/r/bigquery/comments/3cej2b/17_billion_reddit_comments_loaded_on_bigquery/?st=jae26i99&sh=3d53e174) them on Google Big Query. This is cost prohibitive at the moment, so I might have to use Shamu. 

For the purposes of this of project, we shall only use 1 month's worth of data as a sample, and create a way to "read-in" the big data into small data.

## Data-Buffering (SQLite3) 

Since a single month's worth of data is very big, we're going to have to prune a lot of irrelevant data for the purposes of our application.

### Feature Descriptions:

Here are all the features given to for each data point (a comment in a subreddit:

{Author, link_id, score, body, score_hidden (boolean), author_flair_text, gilded, subreddit, edited (boolean), author_flair_css_class, retrieved_on, name, created_utc, parent_id, controversiality, ups, distinguished, id, subreddit_id, downs, archived (boolean).}

(vague in what some of these do)

We're going to use the body, comment_id, and parent_id as the response to the closest match to a user's statement. Further, we'll use votes and gilded as a way of filtering "irrelevant" (in some vague sense) comments.

In the future, we'll also be interested in filtering responses via sub-reddits.

### The
We're first need a database that stores a duple (comment, reply). The reason why is because these files are too big for us to just like read into RAM and then create the training files from a month basis (Reddit has many users). Thus,

But chances are you're gonna want to eventually if you wanted to create a really

Nice chat bot you're gonna be wanting to work on many months of data

so maybe possibly billions of comments you do have that your disposal so when that's the case we

Probably want to have some sort of database now for the purposes here. Just to keep things simple

## Code

We will be using sqlite3 for our database, json to load in the lines from the datadump, and then datetime really just for logging. This wont be totally necessary.

Data is stored as JSON data dumps, named by year and month (YYYY-MM). They are compressed in .bz2.

In [2]:
import sqlite3
import json
from datetime import datetime

In [3]:
#month of data we're analyzing
timeframe = '2015-05'


#We wish to supply a list of actions to commit all at once, because committing to prune a large dataset sequentially is more costly.

sql_transaction = []



#First, we need to establish a connection and cursor. This is true with both SQLite and MySQL.
connection = sqlite3.connect('{}.db'.format(timeframe))
c = connection.cursor()

#Used for the main body of code.
start_row = 0
cleanup = 1000000

Next, we want to make our table. With SQLite, the database is created with the connect if it doesn't already exist. SQL is a different language and it is common to use all-caps to denote SQL specific commands.

In [4]:
#We wish to "execute" an SQL command to make 
def create_table():
    c.execute("CREATE TABLE IF NOT EXISTS parent_reply(parent_id TEXT PRIMARY KEY, comment_id TEXT UNIQUE, parent TEXT, comment TEXT, subreddit TEXT, unix INT, score INT)")

#Function to tokenize newline, return into newlinechar and the double quotes into single quotes.
def format_data(data):
    data = data.replace('\n',' newlinechar ').replace('\r',' newlinechar ').replace('"',"'")
    return data

#Find parent id for the parent data.
def find_parent(pid):
    try:
        sql = "SELECT comment FROM parent_reply WHERE comment_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else: return False
    except Exception as e:
        #print(str(e))
        return False

#Find score from PID.
def find_existing_score(pid):
    try:
        sql = "SELECT score FROM parent_reply WHERE parent_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else: return False
    except Exception as e:
        #print(str(e))
        return False
    
#Next, many comments are either deleted or removed, but also some comments are very long, or very short. We want to make sure comments are of an acceptable length for training, and that the comment wasn't removed or deleted:
def acceptable(data):
    if len(data.split(' ')) > 1000 or len(data) < 1:
        return False
    elif len(data) > 32000:
        return False
    elif data == '[deleted]':
        return False
    elif data == '[removed]':
        return False
    else:
        return True


#Now we define our inject function.
def sql_insert_replace_comment(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """UPDATE parent_reply SET parent_id = ?, comment_id = ?, parent = ?, comment = ?, subreddit = ?, unix = ?, score = ? WHERE parent_id =?;""".format(parentid, commentid, parent, comment, subreddit, int(time), score, parentid)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))
"""
That covers a situation where a comment is already paired with a parent, but we also need to cover comments that don't have parents (but might be a parent to another comment!) 
and comments that do have parents and those parents don't already have a reply. We can further build out insertion block:
"""

def sql_insert_has_parent(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, parent, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}","{}",{},{});""".format(parentid, commentid, parent, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))

def sql_insert_no_parent(commentid,parentid,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}",{},{});""".format(parentid, commentid, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))
        
        
"""
Finally, the last part of our code that we need now is that we need to build the transaction_bldr function.
This function is used to build up insertion statements and commit them in groups, rather than one-by-one. Doing it this way will be much much quicker:
"""



def transaction_bldr(sql):
    global sql_transaction
    sql_transaction.append(sql)
    if len(sql_transaction) > 1000:
        c.execute('BEGIN TRANSACTION')
        for s in sql_transaction:
            try:
                c.execute(s)
            except:
                pass
        connection.commit()
        sql_transaction = []



In [5]:

filespace = '/home/paperspace/reddit_comment_dumps/RC_{}'

    
if __name__ == '__main__':
    #Initiate empty table.
    create_table()
    #initialize counter for progress row_counter will tell just how far we're iterating
    #paired_rows will tell us how many comment+reply we've paired.
    row_counter = 0
    paired_rows = 0

    #Begin buffering data while keeping track of progress overtime
    #with open('J:/chatdata/reddit_data/{}/RC_{}'.format(timeframe.split('-')[0],timeframe), buffering=1000) as f:
    with open(filespace.format(timeframe), buffering=1000) as f:
        for row in f:
            #print(row)
            #time.sleep(555)
            row_counter += 1

            #Next we wish to "read-in" the rows, which is in json format
            if row_counter > start_row:
                try:
                    row = json.loads(row)
                    parent_id = row['parent_id'].split('_')[1]
                    body = format_data(row['body'])
                    created_utc = row['created_utc']
                    score = row['score']
                    
                    comment_id = row['id']
                    
                    subreddit = row['subreddit']
                    parent_data = find_parent(parent_id)
                    
                    existing_comment_score = find_existing_score(parent_id)
                    if existing_comment_score:
                        if score > existing_comment_score:
                            if acceptable(body):
                                sql_insert_replace_comment(comment_id,parent_id,parent_data,body,subreddit,created_utc,score)
                                
                    else:
                        if acceptable(body):
                            if parent_data:
                                if score >= 2:
                                    sql_insert_has_parent(comment_id,parent_id,parent_data,body,subreddit,created_utc,score)
                                    paired_rows += 1
                            else:
                                sql_insert_no_parent(comment_id,parent_id,body,subreddit,created_utc,score)
                except Exception as e:
                    print(str(e))
                            
            if row_counter % 100000 == 0:
                print('Total Rows Read: {}, Paired Rows: {}, Time: {}'.format(row_counter, paired_rows, str(datetime.now())))
            """
            If you're training much larger datasets, you may find there is significant bloat that we need to handle for. 
            This is because only about 10% of the comments are getting paired, 
            so a large % of our database is not actually going to be used. I use the following additional code:
            Directly below the other counter. This requires a new cleanup variable, 
            which specifies how many rows before you "cleanup." 
            
            This will remove bloat to our database and keep insertion speeds fairly high. 
            Each "cleanup" seems to cost about 2K pairs, pretty much wherever you put it. 
            If it's every 100K rows, that'll cost you 2K pairs per 100K rows. I went with 1 million. 
            Another option you have is to clean every 1 million rows, but clean not the last 1 million, 
            but instead the last -1,100,000 to the -100,000th row, since it seems those 2K pairs are happening in the last 100K. 
            Even with this though, you will still lose some pairs. 
            I felt like 2K pairs, out of 100K pairs per 1 million rows was negligible and not important. 
            I also added a start_row variable, so I could start and stop database inserting while trying to improve the speeds 
            a bit. The c.execute("VACUUM") is an SQL command to shrink the size of the database down to what it ought to me. 
            This actually probably isn't required, and you might want to only do this at the very end. 
            I didn't test how long this operation takes. 
            I mostly just did it so I could see immediately after a delete what the size of the database was. 
            """
            if row_counter > start_row:
                if row_counter % cleanup == 0:
                    print("Cleanin up!")
                    sql = "DELETE FROM parent_reply WHERE parent IS NULL"
                    c.execute(sql)
                    connection.commit()
                    c.execute("VACUUM")
                    connection.commit()

FileNotFoundError: [Errno 2] No such file or directory: '/home/paperspace/reddit_comment_dumps/RC_2015-05'